In [1]:
!pip install ultralytics


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from ultralytics import YOLO
import torch

### Clear GPU Memory Before Starting Training

#### This ensures we start with clean memory and no memory leaks.

In [3]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()     # Releases unreferenced cached memory
    torch.cuda.ipc_collect()     # Cleans up any interprocess memory from dead processes

### Load the Pretrained YOLOv8 Model

#### You can choose 'yolov8s.pt' for smaller models or 'yolov8m.pt' for balanced accuracy.

In [4]:
model = YOLO('yolov8m.pt')

### Automatically Select the Best Available Device

#### This ensures that if a CUDA-capable GPU (like your L40S) is available, it's used.

In [5]:
device = 0 if torch.cuda.is_available() else 'cpu'
print(f"Device in use: {device}")
model.to(device)

Device in use: 0


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_

### Begin Training the Model with Optimized Settings

#### These parameters are chosen to improve convergence, utilize your hardware well,
#### and apply techniques like mixed precision and EMA for stability and performance.

In [6]:
results = model.train(
    # Dataset
    data='/teamspace/studios/this_studio/data/yolo-extracted/data.yaml',         # Path to dataset config file (classes, train/val paths, etc.)

    # Training Setup
    epochs=200,                       # Total training cycles (more epochs = better performance if not overfitting)
    imgsz=640,                        # Higher input resolution can help with small objects (adjust batch size if needed)
    batch=16,                         # Number of samples per GPU iteration (balance with VRAM)
    cache=True,                       # Load entire dataset into RAM (you have 128 GB, so it speeds things up)

    # Hardware Optimization
    workers=8,                        # Number of dataloader subprocesses (8–12 works well with 16 vCPUs)
    amp=True,                         # Mixed precision training (faster, less memory, same accuracy)
    device=device,                    # Use GPU if available, otherwise fallback to CPU

    # Learning Rate & Optimizer
    lr0=0.005,                        # Initial learning rate
    lrf=0.0005,                       # Final learning rate (decayed over time)
    warmup_epochs=5,                  # Start with slower learning rate to stabilize early training
    warmup_momentum=0.75,             # Momentum during warmup phase
    optimizer='AdamW',                # Weight-decay-aware Adam optimizer (generally better for generalization)
    weight_decay=0.0005,              # L2 regularization (prevents overfitting)

    # Validation
    val=True,                         # Run validation after every epoch to monitor performance

    # Model Saving & Early Stopping
    save_period=5,                    # Save model every 5 epochs
    patience=15,                      # Stop early if no improvement in validation loss for 15 epochs

    # Additional Optimization
    cos_lr=True,                      # Use cosine decay for smoother convergence to minima
    name='yolov8m-unBalanced-Dataset' # Name of the model
)

New https://pypi.org/project/ultralytics/8.3.131 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.114 🚀 Python-3.10.10 torch-2.2.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/teamspace/studios/this_studio/data/yolo-extracted/data.yaml, epochs=200, time=None, patience=15, batch=16, imgsz=640, save=True, save_period=5, cache=True, device=0, workers=8, project=None, name=yolov8m-unBalanced-Dataset6, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, e

Model summary: 169 layers, 25,859,794 parameters, 25,859,778 gradients, 79.1 GFLOPs

Transferred 469/475 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 186MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.1±0.0 ms, read: 742.2±1068.1 MB/s, size: 161.6 KB)


train: Scanning /teamspace/studios/this_studio/data/yolo-extracted/labels/train.cache... 5931 images, 6 backgrounds, 0 corrupt: 100%|██████████| 5931/5931 [00:00<?, ?it/s]


...

    124/200      7.08G     0.1786     0.1393      0.852         22        640: 100%|██████████| 371/371 [01:49<00:00,  3.40it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 33/33 [00:06<00:00,  5.30it/s]

                   all       1050       1116      0.911      0.959      0.959      0.835
EarlyStopping: Training stopped early as no improvement observed in last 15 epochs. Best results observed at epoch 109, best model saved as best.pt.
To update EarlyStopping(patience=15) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



124 epochs completed in 3.987 hours.
Optimizer stripped from runs/detect/yolov8m-unBalanced-Dataset6/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/yolov8m-unBalanced-Dataset6/weights/best.pt, 52.0MB

Validating runs/detect/yolov8m-unBalanced-Dataset6/weights/best.pt...
Ultralytics 8.3.114 🚀 Python-3.10.10 torch-2.2.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 92 layers, 25,843,234 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 33/33 [00:06<00:00,  5.09it/s]


                   all       1050       1116      0.925      0.948      0.961      0.836
     Accepted_aluminum        105        105      0.929          1      0.994        0.9
      Accepted_plastic        378        378      0.981      0.976      0.992      0.898
            Background         39         39      0.975      0.993      0.994      0.995
                  Hand         84         84      0.729      0.786      0.823      0.576
     Rejected_aluminum        222        222      0.986      0.975      0.992      0.827
      Rejected_plastic        288        288      0.947      0.955      0.972      0.821
Speed: 0.1ms preprocess, 3.6ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/yolov8m-unBalanced-Dataset6


In [ ]:
results = model.train(
    # Dataset
    data='/teamspace/studios/this_studio/data/yolo-extracted-balanced/data.yaml',         # Path to dataset config file (classes, train/val paths, etc.)

    # Training Setup
    epochs=200,                       # Total training cycles (more epochs = better performance if not overfitting)
    imgsz=640,                        # Higher input resolution can help with small objects (adjust batch size if needed)
    batch=16,                         # Number of samples per GPU iteration (balance with VRAM)
    cache=True,                       # Load entire dataset into RAM (you have 128 GB, so it speeds things up)

    # Hardware Optimization
    workers=8,                        # Number of dataloader subprocesses (8–12 works well with 16 vCPUs)
    amp=True,                         # Mixed precision training (faster, less memory, same accuracy)
    device=device,                    # Use GPU if available, otherwise fallback to CPU

    # Learning Rate & Optimizer
    lr0=0.005,                        # Initial learning rate
    lrf=0.0005,                       # Final learning rate (decayed over time)
    warmup_epochs=5,                  # Start with slower learning rate to stabilize early training
    warmup_momentum=0.75,             # Momentum during warmup phase
    optimizer='AdamW',                # Weight-decay-aware Adam optimizer (generally better for generalization)
    weight_decay=0.0005,              # L2 regularization (prevents overfitting)

    # Validation
    val=True,                         # Run validation after every epoch to monitor performance

    # Model Saving & Early Stopping
    save_period=5,                    # Save model every 5 epochs
    patience=15,                      # Stop early if no improvement in validation loss for 15 epochs

    # Additional Optimization
    cos_lr=True,                      # Use cosine decay for smoother convergence to minima
    name='yolov8m-Balanced-Dataset' # Name of the model
)

New https://pypi.org/project/ultralytics/8.3.162 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.114 🚀 Python-3.10.10 torch-2.2.1+cu121 CUDA:0 (NVIDIA L40S, 45596MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/teamspace/studios/this_studio/data/yolo-extracted-balanced/data.yaml, epochs=200, time=None, patience=15, batch=16, imgsz=640, save=True, save_period=5, cache=True, device=0, workers=8, project=None, name=yolov8m-Balanced-Dataset7, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks

Model summary: 169 layers, 25,859,794 parameters, 25,859,778 gradients, 79.1 GFLOPs

Transferred 469/475 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3468.9±1284.0 MB/s, size: 173.3 KB)


train: Scanning /teamspace/studios/this_studio/data/yolo-extracted-balanced/labels/train.cache... 9367 images, 0 backgrounds, 0 corrupt: 100%|██████████| 9367/9367 [00:00<?, ?it/s]


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (7.2GB RAM): 100%|██████████| 9367/9367 [00:07<00:00, 1321.98it/s]


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    139/200      7.08G     0.1089    0.09495     0.8386         16        640: 100%|██████████| 586/586 [01:04<00:00,  9.12it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 52/52 [00:05<00:00,  9.54it/s]

                   all       1652       1918      0.995      0.998      0.995      0.956



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    140/200      7.08G     0.1088    0.09139     0.8356         45        640:  11%|█         | 65/586 [00:07<01:04,  8.13it/s]

### Evaluate the model

In [ ]:
metrics = model.val(data='/teamspace/studios/this_studio/data/yolo-extracted-balanced/data.yaml')  # Path to the validation dataset YAML file

Ultralytics 8.3.114 🚀 Python-3.10.10 torch-2.2.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 92 layers, 25,843,234 parameters, 0 gradients, 78.7 GFLOPs


FileNotFoundError: 'data.yaml' does not exist

### Export the trained model

In [ ]:
model.export(format="onnx")  # or use format="engine" for TensorRT